#Arrange the song using RNN(LSTM)
### 신경망으로 악보 학습 후 새로운 곡 생성.
#### ABC표기(계이름, 박자: 2채널)를 수행한 데이터를 이용하겠습니다.
#### 분류 문제로 해결하였습니다.

##### 먼저, 데이터를 준비하겠습니다.

In [1]:
import music21 #music21 Library. Can play the marked data

# "Little Star". C: do, D: re / 2: half note, 4: note
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2" #나중에 분할을 위해 ' '띄어쓰기를 해줍니다.
#music21.converter.parse(little_star).show('mid') #play

In [2]:
import numpy as np

# 딕셔너리 자료구조로 계이름과 숫자를 변형하는 기준을 세웠습니다.
note2num = {'c':1, 'd':2, 'e':3, 'f':4, 'g':5, 'a':6, 'b':7}
num2note = {1:'c', 2:'d', 3:'e', 4:'f', 5:'g', 6:'a', 7:'b'}

In [3]:
#abc data -> sequential data
def abc2timeseries(s): #s: little_star
  notes=s.split(' ')[2:]
  seq=[]
  for i in notes: #note: ex)c4
    seq.append([note2num[i[0]], int(i[1])])
  return seq

#sequential data -> abc data
def timeseries2abc(t): #t: ex)14
  s='tinynotation: 4/4'
  for i in t:
    s=s+' '+num2note[i[0]]+str(i[1])
  return s

#sequential data -> training set
def seq2dataset(seq,window,horizon):
  X=[]
  Y=[]
  for i in range(len(seq)-(window+horizon)+1):
    x=seq[i:(i+window)]
    X.append(x)
    y=(seq[i+window+horizon-1])
    Y.append(y)
  return np.array(X), np.array(Y)

In [4]:
w=8
h=1

seq=abc2timeseries(little_star)
X,Y=seq2dataset(seq,w,h)
#check data
print(X.shape, Y.shape)
print(X[0],Y[0])

#one-hot table
onehot=[[1,2],[2,2],[3,3],[4,2],[5,2],[6,2],[7,2],[1,4],[2,4],[3,4],[4,4],[5,4],[6,4],[7,4],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8],[7,8]]
#label to one-hot
def to_onehot(l): #l: ex)14
  t=[]
  for i in range(len(l)):
    a=np.zeros(len(onehot))
    a[onehot.index(list(l[i]))] = 1.0
    t.append(a)
  return np.array(t)

split=int(len(X)*1.0)
x_train=X[0:split]
y_train=Y[0:split]
y_train=to_onehot(y_train)

(34, 8, 2) (34, 2)
[[1 4]
 [1 4]
 [5 4]
 [5 4]
 [6 4]
 [6 4]
 [5 2]
 [4 4]] [4 4]


##### 이어서 모델을 설계하고 학습하겠습니다.

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
import tensorflow as tf

#LSTM model
model = Sequential()
model.add(LSTM(units=128, activation='relu', input_shape=x_train[0].shape))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=200, batch_size=1, verbose=2)

Epoch 1/200
34/34 - 2s - loss: 2.8960 - accuracy: 0.1176 - 2s/epoch - 61ms/step
Epoch 2/200
34/34 - 0s - loss: 2.2519 - accuracy: 0.1765 - 200ms/epoch - 6ms/step
Epoch 3/200
34/34 - 0s - loss: 2.0954 - accuracy: 0.2353 - 194ms/epoch - 6ms/step
Epoch 4/200
34/34 - 0s - loss: 2.0146 - accuracy: 0.2353 - 194ms/epoch - 6ms/step
Epoch 5/200
34/34 - 0s - loss: 1.9370 - accuracy: 0.3235 - 169ms/epoch - 5ms/step
Epoch 6/200
34/34 - 0s - loss: 1.8096 - accuracy: 0.2647 - 163ms/epoch - 5ms/step
Epoch 7/200
34/34 - 0s - loss: 1.6456 - accuracy: 0.5000 - 174ms/epoch - 5ms/step
Epoch 8/200
34/34 - 0s - loss: 1.6247 - accuracy: 0.3824 - 150ms/epoch - 4ms/step
Epoch 9/200
34/34 - 0s - loss: 1.5238 - accuracy: 0.4118 - 94ms/epoch - 3ms/step
Epoch 10/200
34/34 - 0s - loss: 1.5418 - accuracy: 0.3824 - 105ms/epoch - 3ms/step
Epoch 11/200
34/34 - 0s - loss: 1.2598 - accuracy: 0.4412 - 117ms/epoch - 3ms/step
Epoch 12/200
34/34 - 0s - loss: 1.3966 - accuracy: 0.4706 - 105ms/epoch - 3ms/step
Epoch 13/200
34/

#####학습이 완료되었습니다. 이를 바탕으로 편곡을 진행하겠습니다.

In [6]:
def arranging_music(model, first_measure, duration): #first_measure: 첫 소절, duration: 생성될 곡의 길이
  music=first_measure
  for i in range(duration):
    p=model.predict(np.float32(np.expand_dims(music[-w:],axis=0)))
    music=np.append(music,[onehot[np.argmax(p)]],axis=0)
  return timeseries2abc(music)

new_song = arranging_music(model, x_train[0], 50)

In [7]:
print(new_song)

tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4
